# Chapter 1

### Pandas postgresql

```
import pandas as pd
import psycopg2

def execute_query(sql_query, dbname='hr', user='postgres', password='postgres', port='5432'):
    # Create a connection to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, port=port)

    # Use read_sql to execute the query and load the results into a DataFrame
    df = pd.read_sql(sql_query, conn)

    # Close the database connection
    conn.close()

    # Return the DataFrame
    return df

query_result = execute_query('SELECT * FROM jobs')
print(query_result)
```

### Column data type sql

```
SELECT COLUMN_NAME,
       DATA_TYPE,
       IS_NULLABLE,
       CHARACTER_MAXIMUM_LENGTH,
       NUMERIC_PRECISION,
       NUMERIC_SCALE
FROM INFORMATION_SCHEMA.COLUMNS -- INFORMATION_SCHEMA.TABLES for table information
WHERE TABLE_NAME='your_table_name'
 	AND TABLE_SCHEMA = 'yourSchemaName';
```

### Array in SQL

```
-- Creating table that has a column to hold arrays
CREATE TABLE grades (
student_id int,
email text[][],
test_scores int[]
);

-- Insering array values in the table
INSERT INTO grades
VALUES (1, 
'{{"work","work1@datacamp.com"},{"other","other1@datacamp.com"}}',
'{92,85,96,88}' );

-- Querying array
SELECT email[2][1] as type, email[2][2] as address, test_scores[1]
FROM grades
-- NOTICE :  ANY (email) = 'other' is not the right syntax, it should be 'other' = ANY (email)
WHERE 'other' = ANY (email); -- Alternative: WHERE email @> ARRAY['other'];
```

# Chapter 2

### SQL datetime

```


-- DATE operations (Advised : Use TIMESTAMP FORMAT instead of DATE FORMAT to acquire preciseness)
SELECT date '2005-09-11' - date '2005-09-10'; -- Result will be an integer as no of days (1)
SELECT date '2005-09-11' + integer '3'; -- Result will be a date (2005-09-14)

-- TIMESTAMP operations
SELECT date '2005-09-11 00:00:00' - date '2005-09-09 12:00:00'; -- Result will be an interval (1 day 12:00:00)
SELECT AGE(timestamp '2005-09-11 00:00:00', timestamp '2005-09-09 12:00:00'); -- Result will be an interval (1 day 12:00:00)
SELECT timestamp '2019-05-01' + 21 * INTERVAL '1 day';  -- Result will be a new timestamp (2019-05-22 00:00:00)
SELECT rental_date + INTERVAL '3 days' AS expected_return_date -- adding timestamp with interval = new timestamp
FROM rental;

-- Current Timestamp
SELECT NOW()::timestamp;    -- casting will cut the timezone information
SELECT CAST(NOW() as timestamp);
SELECT CURRENT_TIMESTAMP(2); -- control precision
SELECT CURRENT_DATE; -- current date
SELECT CURRENT_TIME; -- current time with timezone information

-- Extract date and time information

SELECT EXTRACT (month FROM timestamp '2005-01-24 05:12:00') AS month; -- Result will be month (1)
SELECT DATE_PART('month', timestamp '2005-01-24 05:12:00') AS month; -- Result will be month (1)
SELECT DATE_TRUNC('year', TIMESTAMP '2005-05-21 15:30:30'); -- Result will truncate to specified precision (2005-01-01 00:00:00)
```

# Chapter 3

### String Operations

```
SELECT CONCAT(col_A,' ', col_B) AS concat_col FROM table_name; -- Concat 2 columns
SELECT col_A || ' ' || col_B AS concat_col FROM table_name; -- Concat 2 columns

SELECT UPPER(col) FROM table_name; -- Turning a column into all uppercase
SELECT LOWER(col) FROM table_name; -- Turning a column into all lowercase
SELECT LEFT(col, length) FROM table_name; -- Take n no of characters from the left side of the column
SELECT RIGHT(col, length) FROM table_name; -- Take n no of characters from the right side of the column
-- Extracting Substring
SELECT SUBSTRING(col, starting_index, length) FROM table_name; -- starting_index, length are integer values
SELECT SUBSTRING(col FROM starting_index FOR POSITION('@' IN col)) FROM table_name; -- starting_index, length are integer values
SELECT SUBSTR(col, starting_index, length) FROM table_name; -- starting_index, length are integer values
SELECT REVERSE(col) FROM table_name; -- Reverse a string
SELECT CHAR_LENGTH(col) FROM table_name;  -- length of a string column
SELECT LENGTH(col) FROM table_name; -- length of a string column
SELECT POSITION('@' IN col) FROM table_name; -- position of a character in the string
SELECT STRPOS(email, '@') FROM table_name; -- position of a character in the string
SELECT REPLACE(col, 'old string', 'new string') FROM table_name; -- replacing characters
SELECT LPAD('padded', total_string_size, '#'); -- appending character from the left (####padded)
SELECT RPAD('padded', total_string_size, '#'); -- appending character from the right (padded####)
-- TRIM([leading | trailing | both] [characters] from string)
SELECT TRIM(BOTH 'x' FROM 'xxxxHelloxxxx') -- Hello
SELECT TRIM(TRAILING 'x' FROM 'xHelloxxxx') -- xHello
SELECT TRIM(LEADING 'x' FROM 'xxxxHellox') -- Hellox

-- Pattern matching
SELECT * FROM table_name WHERE col LIKE '_ELF%'; -- Case sensitive match
SELECT * FROM table_name WHERE col ILIKE '_ELF%'; -- Case insensitive match
SELECT * FROM table_name WHERE to_tsvector(col) @@ to_tsquery('elf'); -- Case insensitive match

CREATE EXTENSION IF NOT EXISTS fuzzystrmatch;
SELECT levenshtein('GUMBO', 'GAMBOL'); -- levenshtein score of similarity between 2 words
CREATE EXTENSION IF NOT EXISTS pg_trgm;
SELECT similarity('GUMBO', 'GAMBOL'); -- trigram similarity score

```

# Chapter 4

### SQL Custom Data Type

```
-- Create table with custom ENUM data type
CREATE TYPE mood AS ENUM ('happy', 'sad', 'angry', 'excited'); -- custom ENUM data type
CREATE TABLE person ( -- table creation
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    current_mood mood
);
INSERT INTO person (name, current_mood) VALUES ('Alice', 'happy'); -- data insertion

-- Create other custom data type
CREATE TYPE phone_number AS ( -- custom type
    country_code text,
    area_code text,
    number text
);
CREATE TABLE contacts ( -- creating table
    contact_id serial PRIMARY KEY,
    name text,
    phone phone_number
);
INSERT INTO contacts (name, phone) VALUES   -- data insertion
  ('John Doe', ROW('1', '555', '1234567')),
  ('Jane Smith', ROW('1', '555', '9876543'));

SELECT name, (phone).number AS number -- query table with custom data type 
FROM contacts;

-- See user defined data types
SELECT typname, typcategory
FROM pg_type

-- See using information schema
SELECT COLUMN_NAME,
       DATA_TYPE,
       UDT_NAME,
       IS_NULLABLE,
       CHARACTER_MAXIMUM_LENGTH,
       NUMERIC_PRECISION,
       NUMERIC_SCALE
FROM INFORMATION_SCHEMA.COLUMNS -- INFORMATION_SCHEMA.TABLES for table information
WHERE TABLE_NAME='your_table_name';
```

### Custom Function

```
-- Function that returns square of a given parameter
CREATE FUNCTION func_name(i integer) RETURNS integer AS 
$$
    BEGIN
        RETURN i * i; 
    END;
$$ LANGUAGE plpgsql;

-- Calling the function
SELECT func_name(10);
```

### Extensions

```
-- Available extensions ready to be enabled
SELECT name FROM pg_available_extensions;

-- Enable an extension
CREATE EXTENSION IF NOT EXISTS fuzzystrmatch;

-- See enabled extensions
SELECT extname FROM pg_extension;

```